In [58]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
#para el token
import jwt
#para request
import requests
from getpass import getpass
import parametros

### Conexion Elastic Search

In [60]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

### tomando fecha mas reciente del indice

In [61]:
total_docs = 0
try:
    response = es.search(
        index='bmcingest_test3',
        body={"aggs" : {
                   "max_date": {"max": {"field": "fechahora", "format": "yyyyMMddHHmmss"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max=response["aggregations"]["max_date"]['value_as_string']
except:
    fecha_max = '20100101000001'
print(fecha_max)

20100101000001


### Definiendo función de llenado de ES

In [62]:
def filterKeys(document):
        return {key: document[key] for key in use_these_keys }
    
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'bmcingest_test3', 
                #"_id": f"{document['tipodoc']+document['documento']}",
                "_source": filterKeys(document),
            }

### Solicitud de token

In [63]:
payload = json.dumps({
  "username": parametros.bmc_username,
  "password": parametros.bmc_password,
  "tenantName": "*"
})
headers = {
  'Content-Type': 'application/json'
}

r_tk = requests.request("POST", parametros.bmc_url_token, headers=headers, data=payload, verify=False)
if r_tk.status_code == 200:
    res = json.loads(r_tk.text)
    #print(res['response']['authToken'])
else:
    print("el else saca esto: ",r_tk.status_code)
    #exit()

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mintictsps.triara.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [64]:
autoriza = 'authtoken ' + res['response']['authToken']
payload = json.dumps({
  "className": "EYE_EVENT",
  "criteria": {
    "value": "CRITICAL",
    "identifier": "severity",
    "operator": "EQUALS"
  }
})
headers = {
  'Authorization' : autoriza,
  'Content-Type': 'application/json'
}

r = requests.request("POST", parametros.bmc_url, headers=headers, data=payload, verify=False)
if r.status_code == 200:
    res_api = json.loads(r.text)
    #dato_param = res['paging']
    datos_api = pd.DataFrame(columns=['mc_modhist','mc_location','mc_client_address','itsm_incident_id'
                                      ,'date','CLASS','mc_original_severity','mc_host','severity'
                                      ,'itsm_incident_id','itsm_incident_status'])
    contador=1
    for filas in res_api['responseList']:
        for atributos in filas['eventList']:
            #print(atributos['attributes']['date'])
            datos_api.loc[contador] = atributos['attributes']
            contador+=1
            #break
    try:
        datos_api.fillna('', inplace=True)
        datos_api['mc_modhist'] = datos_api['mc_modhist'].str[1:-1]
        datos_api['fechahora'] = datos_api["date"].str.split(".", n = 1, expand = True)[0]
        datos_api.drop(columns=['date'], inplace=True)
    except:
        #pass
        print("error al leer el dataframe")
        exit()
     #datos_api = datos_api.drop(datos_api[(datos_api['fechahora']<= fecha_max)].index)
    now = datetime.now()
    format_ES = "%Y.%m.%d"
    fecha_hoy = str(now.strftime(format_ES))
    indice = 'bmcingest_test3' #+ '-' +fecha_hoy
    # Quitado el fecha_hoy solicitado por Napoleon  [GM]

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mintictsim.triara.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [65]:
datos_api

,mc_modhist,mc_location,mc_client_address,itsm_incident_id,CLASS,mc_original_severity,mc_host,severity,itsm_incident_id,itsm_incident_status,fechahora
1,entuity,48507-ZZZY842,172.27.216.171,,EYE_EVENT,CRITICAL,172.25.60.169,CRITICAL,,,20211008013541
2,entuity,26672-ZGYO387,172.27.216.171,,EYE_EVENT,CRITICAL,172.25.116.33,CRITICAL,,,20211011020736
3,entuity,48938-ZGYO669,172.27.216.171,,EYE_EVENT,CRITICAL,172.25.31.89,CRITICAL,,,20211011030340
4,entuity,44739-ZGYO228,172.27.216.171,,EYE_EVENT,CRITICAL,172.25.29.41,CRITICAL,,,20211012120746
5,entuity,20760-ZGYO159,172.27.216.171,,EYE_EVENT,CRITICAL,172.28.103.225,CRITICAL,,,20211012171943
...,...,...,...,...,...,...,...,...,...,...,...
196,entuity,70856-VZVF789,100.123.26.213,,EYE_EVENT,CRITICAL,172.25.109.145,CRITICAL,,,20211030045544
197,entuity,67843-VZVF193,100.123.26.213,,EYE_EVENT,CRITICAL,172.25.91.41,CRITICAL,,,20211030064148
198,entuity,36530-ZGYO552,172.27.216.171,,EYE_EVENT,CRITICAL,172.25.59.25,CRITICAL,,,20211030071942
199,entuity,37710-VZVF878,100.123.26.213,,EYE_EVENT,CRITICAL,172.25.168.33,CRITICAL,,,20211030084342


In [22]:
datos_api.to_excel("bmc.xlsx")

### Solicitando los datos a la API

In [66]:
autoriza = 'authtoken ' + res['response']['authToken']
payload = json.dumps({
  "className": "EYE_EVENT",
  "criteria": {
    "value": "CRITICAL",
    "identifier": "severity",
    "operator": "EQUALS"
  }
})
headers = {
  'Authorization' : autoriza,
  'Content-Type': 'application/json'
}

r = requests.request("POST", parametros.bmc_url, headers=headers, data=payload, verify=False)
if r.status_code == 200:
    res_api = json.loads(r.text)
    #dato_param = res['paging']
    datos_api = pd.DataFrame(columns=['mc_modhist','mc_location','mc_client_address','itsm_incident_id'
                                      ,'date','CLASS','mc_original_severity','mc_host','severity'
                                      ,'itsm_incident_id','itsm_incident_status'])
    contador=1
    for filas in res_api['responseList']:
        for atributos in filas['eventList']:
            #print(atributos['attributes']['date'])
            datos_api.loc[contador] = atributos['attributes']
            contador+=1
            #break
    try:
        datos_api.fillna('', inplace=True)
        datos_api['mc_modhist'] = datos_api['mc_modhist'].str[1:-1]
        datos_api['fechahora'] = datos_api["date"].str.split(".", n = 1, expand = True)[0]
        datos_api.drop(columns=['date'], inplace=True)
    except:
        #pass
        print("error al leer el dataframe")
        exit()
    datos_api = datos_api.drop(datos_api[(datos_api['fechahora']<= fecha_max)].index)
    now = datetime.now()
    format_ES = "%Y.%m.%d"
    fecha_hoy = str(now.strftime(format_ES))
    indice = 'bmcingest_test' #+ '-' +fecha_hoy
    # Quitado el fecha_hoy solicitado por Napoleon  [GM]

    use_these_keys = ['mc_modhist','mc_location','mc_client_address','itsm_incident_id'
                          ,'fechahora','CLASS','mc_original_severity','mc_host','severity'
                          ,'itsm_incident_id','itsm_incident_status','@timestamp']
    
    timestamp = datetime.now()
    datos_api['@timestamp'] = timestamp.isoformat()

    salida = helpers.bulk(es, doc_generator(datos_api))
    print("Fecha inserción: ",timestamp," - Documentos insertados: ", salida[0])
else:
    print("error de request: ",r.status_code)
    exit()

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mintictsim.triara.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Fecha inserción:  2021-11-09 12:30:31.444914  - Documentos insertados:  200
